# Construir um Chatbot

Vamos apresentar um exemplo de como projetar e implementar um chatbot alimentado por um LLM. Este chatbot será capaz de ter uma conversa e lembrar interações anteriores. Observe que este chatbot que estamos construindo usará apenas o modelo de linguagem para manter uma conversa. Existem vários outros conceitos relacionados que você pode estar procurando:

- Conversacional RAG: Habilitar uma experiência de chatbot sobre uma fonte externa de dados
- Agentes: Construir um chatbot que possa realizar ações
Este tutorial cobrirá o básico, que será útil para esses dois tópicos mais avançados, mas sinta-se à vontade para ir diretamente a eles, se preferir.

Aqui estão alguns dos componentes de alto nível com os quais trabalharemos:

- Chat Models. A interface do chatbot é baseada em mensagens em vez de texto bruto, sendo mais adequada para Modelos de Chat do que para LLMs de texto.
- Prompt Templates, que simplificam o processo de montagem de prompts que combinam mensagens padrão, entrada do usuário, histórico do chat e (opcionalmente) contexto adicional obtido.
- Chat History, que permite ao chatbot "lembrar" interações passadas e levá-las em consideração ao responder a perguntas de acompanhamento.
- Depuração e rastreamento de sua aplicação usando LangSmith

Vamos cobrir como integrar os componentes acima para criar um chatbot conversacional poderoso.

### Instalação

In [3]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


### LangSmith

Muitas das aplicações que você construir com LangChain conterão múltiplas etapas com várias invocações de chamadas de LLM. À medida que essas aplicações se tornam mais complexas, torna-se crucial a capacidade de inspecionar exatamente o que está acontecendo dentro da sua cadeia ou agente. A melhor maneira de fazer isso é com LangSmith.

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


### Escolha do modelo

Primeiramente, vamos aprender como usar um modelo de linguagem por si só. LangChain suporta vários modelos de linguagem diferentes que você pode usar de forma intercambiável.

In [5]:
pip install langchain-mistralai

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_mistralai import ChatMistralAI

os.environ["MISTRAL_API_KEY"] = getpass.getpass()
model = ChatMistralAI(model="mistral-large-latest")

 ········


Primeiro, vamos usar o modelo diretamente. Os ChatModels são instâncias de "Runnables" do LangChain, o que significa que eles expõem uma interface padrão para interagir com eles. Para simplesmente chamar o modelo, podemos passar uma lista de mensagens para o método .invoke.

In [9]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! It's nice to meet you. How can I assist you today?", response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 27, 'completion_tokens': 18}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-42dfd1dd-4e92-4ca5-87bd-63df4afeb39f-0')

O modelo por si só não tem nenhum conceito de estado. Por exemplo, se você fizer uma pergunta de acompanhamento:

In [10]:
model.invoke([HumanMessage(content="Qual o meu nome?")])

AIMessage(content='Infelizmente, eu não tenho acesso a informações pessoais sobre você, como seu nome, a menos que você me informe. Se você quiser me dizer o seu nome, ficarei feliz em me lembrar durante nossa conversa.', response_metadata={'token_usage': {'prompt_tokens': 9, 'total_tokens': 76, 'completion_tokens': 67}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-c8db9be9-deed-4dfb-88ba-daff8bbe86a8-0')

Podemos ver que ele não considera o turno anterior da conversa como contexto e não consegue responder à pergunta. Isso resulta em uma experiência de chatbot terrível!

Para contornar isso, precisamos passar todo o histórico da conversa para o modelo. Vamos ver o que acontece quando fazemos isso:

In [11]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Olá! eu sou o João"),
        AIMessage(content="Oi João! Como você está se sentindo hoje?"),
        HumanMessage(content="Qual o meu nome?"),
    ]
)

AIMessage(content='Seu nome é João. Como posso ajudar você hoje, João?\n\nEstou aqui para responder suas perguntas e fornecer informações úteis sobre uma variedade de tópicos. Se tiver alguma dúvida ou precisar de ajuda com algo, sinta-se à vontade para perguntar!\n\nPor exemplo, posso ajudar a responder perguntas sobre notícias atuais, informações gerais, esportes, entretenimento, tecnologia, educação, saúde e muito mais. Se tiver alguma dúvida ou precisar de ajuda com algo, sinta-se à vontade para perguntar!\n\nEstou aqui para ajudar você de todas as maneiras possíveis. Então, sinta-se à vontade para fazer qualquer pergunta que você tenha em mente, e farei o meu melhor para responder com precisão e clareza.', response_metadata={'token_usage': {'prompt_tokens': 35, 'total_tokens': 284, 'completion_tokens': 249}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-2c5d5388-7140-4d8b-936b-bc5ff4dee600-0')

E agora podemos ver que obtemos uma boa resposta!

Essa é a ideia básica que sustenta a capacidade de um chatbot de interagir de forma conversacional. Então, como podemos implementar isso da melhor maneira?

### Historico de mensagens

Podemos usar uma classe de Histórico de Mensagens para envolver nosso modelo e torná-lo com estado. Isso manterá o controle das entradas e saídas do modelo e as armazenará em algum banco de dados. Interações futuras então carregarão essas mensagens e as passarão para a cadeia como parte da entrada. Vamos ver como fazer isso!

Primeiro, certifique-se de instalar o langchain-community, pois vamos usar uma integração lá para armazenar o histórico de mensagens.

In [12]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


Depois disso, podemos importar as classes relevantes e configurar nossa cadeia que envolve o modelo e adiciona esse histórico de mensagens. Uma parte crucial aqui é a função que passamos como get_session_history. Espera-se que essa função receba um session_id e retorne um objeto de Histórico de Mensagens. Esse session_id é usado para distinguir entre conversas separadas e deve ser passado como parte da configuração ao chamar a nova cadeia (mostraremos como fazer isso).

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

Agora precisamos criar uma 'config' que passamos para o runnable a cada vez. Essa config contém informações que não fazem parte diretamente da entrada, mas que ainda são úteis. Neste caso, queremos incluir um session_id. Isso deve ficar assim:

In [14]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Olá! Eu sou o Paulo e tenho 25 anos")],
    config=config,
)

response.content

Parent run 79baf7e5-1f22-4925-8f7d-0a857387e082 not found for run 8704ba39-ce1a-4426-958b-061ec386ea5d. Treating as a root run.


'Olá Paulo! É um prazer te conhecer. Então, você tem 25 anos e está pronto para explorar o mundo da programação? Vamos começar essa jornada juntos! Se você tiver alguma dúvida ou precisar de ajuda, estou aqui para te ajudar.'

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="Qual é o meu nome?")],
    config=config,
)

response.content

Parent run 056b1f03-6457-4e43-a7be-55bbc61920b9 not found for run 83f4d3f7-da40-4828-8caf-44f97afb2aae. Treating as a root run.


'Baseado na nossa última conversa, você disse que seu nome é Paulo. Se isso estiver correto, posso confirmar que seu nome é Paulo. Caso contrário, por favor, me informe o seu nome correto para que eu possa me referir a você corretamente.'

Ótimo! Nosso chatbot agora lembra informações sobre nós. Se alterarmos a configuração para referenciar um session_id diferente, podemos ver que ele inicia a conversa do zero.

In [16]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Qual é o meu nome?")],
    config=config,
)

response.content

Parent run b9cd9093-ea0b-4c60-8558-531dd27b99c8 not found for run 508b92bc-1017-4b21-89c6-9aa9f9712644. Treating as a root run.


'Infelizmente, eu não tenho acesso a informações pessoais sobre você, como seu nome, a menos que você mesmo me informe. Se quiser, você pode me dizer o seu nome e eu farei o possível para lembrar durante nossa conversa.'

No entanto, sempre podemos voltar à conversa original (já que estamos persistindo isso em um banco de dados).

In [17]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Eu tenho quantos anos?")],
    config=config,
)

response.content

Parent run dc6f1f93-d904-4ffa-afcc-312d7f692cb4 not found for run 01444d42-8715-4904-80c2-4d7897b77a47. Treating as a root run.


'Na nossa última conversa, você mencionou que tem 25 anos. Se isso ainda estiver correto, posso confirmar que você tem 25 anos. Caso contrário, por favor, me informe sua idade atualizada para que eu possa ter essa informação correta.'

Assim é como podemos suportar um chatbot tendo conversas com muitos usuários!

Por enquanto, tudo o que fizemos foi adicionar uma camada simples de persistência em torno do modelo. Podemos começar a torná-lo mais complicado e personalizado adicionando um modelo de prompt.

#### Prompt templates

Os prompts templates ajudam a transformar informações brutas do usuário em um formato com o qual o LLM pode trabalhar. Neste caso, a entrada bruta do usuário é apenas uma mensagem, que estamos passando para o LLM. Vamos complicar um pouco isso agora. Primeiro, vamos adicionar uma mensagem de sistema com algumas instruções personalizadas (ainda usando mensagens como entrada). Em seguida, vamos adicionar mais entradas além das mensagens.

Primeiro, vamos incluir uma mensagem de sistema. Para fazer isso, criaremos um ChatPromptTemplate. Vamos usar MessagesPlaceholder para passar todas as mensagens.

In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente prestativo. Responda a todas as perguntas da melhor maneira possível."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Observe que isso muda um pouco o tipo de entrada - em vez de passar uma lista de mensagens, agora estamos passando um dicionário com uma chave messages que contém uma lista de mensagens.

In [27]:
response = chain.invoke({"messages": [HumanMessage(content="Olá, sou Pedro")]})

response.content

'Olá Pedro! Estou aqui para ajudá-lo. Sinta-se à vontade para fazer suas perguntas e farei o meu melhor para responder de forma precisa e útil.'

Agora podemos encapsular isso no mesmo objeto Messages History de antes.

In [28]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [29]:
config = {"configurable": {"session_id": "abc5"}}

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="Olá, meu nome é Carlos")],
    config=config,
)

response.content

Parent run 4d2ef429-1534-4bdd-84dc-3767252ebd31 not found for run 8fc6b716-dbbe-4ba3-9fa5-8eb860731be9. Treating as a root run.


"Obrigado por me dizer o seu nome, Carlos! É um prazer fazer sua conhecimento. Estou aqui para responder às suas perguntas da melhor maneira possível, então por favor, sinta-se à vontade para perguntar o que você quiser.\n\nThank you for letting me know your name, Carlos! It's nice to meet you. I'm here to answer your questions to the best of my ability, so please feel free to ask me anything you want."

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="Qual o meu nome?")],
    config=config,
)

response.content

Parent run 388b5081-8627-4f7b-999e-270eb85b1e00 not found for run c4f358fe-3935-486f-a91d-7bfcc7d83e8e. Treating as a root run.


'Sim, Carlos, eu farei meu melhor para responder às suas perguntas da melhor maneira possível.\n\nVocê me disse que o seu nome é Carlos. É isso mesmo, certo? Se eu estiver enganado, por favor me corrija.\n\nYes, Carlos, I will do my best to answer your questions to the best of my ability.\n\nYou told me that your name is Carlos. Is that correct? If I am mistaken, please let me know.'

Incrível! Vamos agora tornar nosso prompt um pouco mais complicado. Vamos supor que o modelo de prompt agora se pareça com isto:

In [39]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente prestativo. Responda a todas as perguntas da melhor maneira possível em {idioma}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Observe que adicionamos uma nova entrada de idioma ao prompt. Agora podemos invocar a cadeia e passar um idioma de nossa escolha.

In [45]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "idioma": "Spanish"}
)

response.content

'¡Hola! ¿Cómo estás? Soy un asistente diseñado para ayudarte. ¿En qué puedo ayudarte hoy?\n\nRecuerda que aunque prefiero hablar en español, también puedo entender y responder en otros idiomas si es necesario. ¿Cómo puedo asistirte hoy?'

Vamos agora encapsular esta cadeia mais complicada em uma classe Message History. Desta vez, porque há múltiplas chaves na entrada, precisamos especificar a chave correta a ser usada para salvar o histórico do chat.

In [46]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [47]:
config = {"configurable": {"session_id": "abc11"}}

In [49]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "idioma": "Spanish"},
    config=config,
)

response.content

Parent run 129679f7-5243-47e2-8768-75c519e6b7f5 not found for run 247f4145-6f0a-4155-9eb8-0fee8faf1f96. Treating as a root run.


'Hola, soy Todd, un asistente dispuesto a ayudarte. Voy a responder a todas tus preguntas de la mejor manera posible en español. ¿En qué puedo ayudarte hoy?'

In [50]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "idioma": "Spanish"},
    config=config,
)

response.content

Parent run 02560950-6134-4178-96a4-2514846bf22b not found for run e14fb415-4ba5-4388-baf5-1df270a7418f. Treating as a root run.


'Basándome en la información que tengo, tu nombre es Todd. Sin embargo, si me estás pidiendo que adivine tu nombre, lamento decirte que no puedo hacerlo porque no tengo la capacidad de adivinar. Si me proporcionas tu nombre, estaré encantado de dirigirme a ti por él.'

### Gerenciar o histórico da conversa

Um conceito importante ao construir chatbots é como gerenciar o histórico da conversa. Se não for gerenciado adequadamente, a lista de mensagens crescerá ilimitadamente e poderá exceder a janela de contexto do LLM (Modelo de Linguagem Grande). Portanto, é importante adicionar um passo que limite o tamanho das mensagens que você está passando.

É crucial fazer isso ANTES do modelo de prompt, mas DEPOIS de carregar as mensagens anteriores do histórico de mensagens.

Podemos fazer isso adicionando um passo simples antes do prompt que modifica a chave das mensagens adequadamente e, em seguida, encapsular essa nova cadeia na classe Message History. Primeiro, vamos definir uma função que modificará as mensagens passadas. Vamos fazer com que ela selecione as k mensagens mais recentes. Em seguida, podemos criar uma nova cadeia adicionando isso no início.

In [51]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

Vamos tentar agora! Se criarmos uma lista de mensagens com mais de 10 mensagens, podemos ver que ele não lembrará mais das informações nas mensagens mais antigas.

In [52]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [53]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "idioma": "Portuguese",
    }
)
response.content

'Desculpa, você ainda não me disse o seu nome. Como posso te chamar?'

Mas se perguntarmos sobre informações que estão nas últimas dez mensagens, ele ainda se lembrará delas.

In [54]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "idioma": "Portuguese",
    }
)
response.content

'Desculpe, não sei qual é o seu sabor favorito de sorvete. É vanila, chocolate, morango ou outro sabor?'

Vamos agora encapsular isso no Message History.

In [55]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [56]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "idioma": "Portuguese",
    },
    config=config,
)

response.content

Parent run 676c3789-6472-4ee2-aa6f-053bb6ed43f3 not found for run eaa94d2b-fece-4d23-9dba-c2995d1ab7cc. Treating as a root run.


'Desculpe, você ainda não me disse o seu nome. Como posso te chamar?'

Agora há duas novas mensagens no histórico de bate-papo. Isso significa que ainda mais informações que estavam acessíveis em nosso histórico de conversas não estão mais disponíveis!

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "idioma": "Portuguese",
    },
    config=config,
)

response.content

Parent run 1cc70a05-1aa7-4c0f-973a-4845a82b78ee not found for run 37f816bd-26fb-4739-bacf-2eba53004c34. Treating as a root run.


'Desculpe, como eu não conheço você pessoalmente, eu não sei qual é o seu sorvete favorito. Eu posso ajudar a encontrar algumas opções populares de sorvete, se você gostaria.'

### streaming

Agora temos uma função chatbot. No entanto, uma consideração UX realmente importante para aplicativos de chatbot é o streaming. LLMs às vezes podem demorar para responder, então, para melhorar a experiência do usuário, muitos aplicativos fazem o streaming de volta de cada token conforme ele é gerado. Isso permite que o usuário veja o progresso.

Na verdade, é muito fácil fazer isso!

Todas as cadeias (chains) expõem um método .stream, e aquelas que usam histórico de mensagens não são diferentes. Podemos simplesmente usar esse método para obter uma resposta em streaming.

In [58]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "idioma": "Portuguese",
    },
    config=config,
):
    print(r.content, end="|")

Parent run 373494a4-3c10-41b6-aac7-70f7b11f5c8e not found for run bf614122-86f9-46aa-9070-3889b30de2fa. Treating as a root run.


|Hello| Todd|!| Sure|,| here|'|s| a| joke| for| you|:|

P|or| que| o| gol|fin|ho| nad|ava| em| volta| do| t|ub|ar|ão|?|

P|or|que| qu|eria| faz|er| um| "|t|ub|ar|ol|ho|"|!|

(|Why| was| the| dol|ph|in| swimming| around| the| sh|ark|?| Because| it| wanted| to| make| a| "|sh|ark|-|ole|"|!)||